In [181]:
import pandas as pd
import dask.dataframe as dd
import json
import re
import numpy as np

In [182]:
df = pd.read_csv("./data/TSLA_2020_2022/features/TSLA_2020_2022_003.csv")
df["features"][0]


'[[-0.09616660106336444 -0.1312780345741137 -0.03355583488812036\n  -0.17013592425237037 -0.4804322624287979 -0.3626722446528914\n  -0.6784775217645621 -0.039478893969622515 -0.03777629695791152\n  0.4573223570190641]\n [10.398620611963786 9.154198538491375 -0.07457118011429675\n  0.5965288127590641 -0.6764925639982782 -0.05592429014938174\n  1.0272537733193632 -0.039478893969622515 -0.03777629695791152\n  0.0121317157712304]\n [-0.09616660106336444 -0.12400752719028164 0.009872177704301691\n  -0.1516906490330972 0.38223306447691524 -0.21919516640717396\n  0.0038149962690080027 -0.015719675058837312 -0.03777629695791152\n  0.41139514731369164]\n [-0.09616660106336444 0.00694150140423166 1.0280178062599743\n  0.4556673031616809 3.0617238525931456 1.078808299842274\n  -0.6784775217645621 -0.039478893969622515 -0.03777629695791152\n  0.5658578856152512]\n [-0.09616660106336444 -0.02677447789222021 -0.15418920320040386\n  0.013261285207784372 -0.807199431711265 0.25602372993357125\n  -0.33

In [183]:
def float_converter(x): return np.array(
    x.translate({ord(c): None for c in "[]"})
    .replace("\n", "")
    .replace("  ", " ")
     .split(" "),
    dtype=float
).reshape(10, -1)


def str_converter(x): return np.array(
    x.translate({ord(c): None for c in "'[]"})
    .replace("\n", "")
    .replace("  ", " ")
     .split(" "))


In [184]:
features_df = dd.read_csv(
    "./data/TSLA_2020_2022/features/" + "*.csv",
    converters={
        "features": float_converter,
        "inference": str_converter,
        "confidence": float_converter,
    }
)

In [185]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    df = features_df.compute()

[########################################] | 100% Completed | 14.9s


In [186]:
df = df.explode(column=['features', 'inference', 'confidence'])
df['confidence'] = df['confidence'].apply(lambda x: x[0])
df['inference'] = df['inference'].apply(lambda x: 1 if x == 'LABEL_1' else -1)


In [187]:
df.head()

,Unnamed: 0,date,features,inference,confidence
0,0,2019-12-24,"[-0.09616660106336444, -0.1308785561464306, -0...",1,0.938756
0,0,2019-12-24,"[-0.09616660106336444, -0.13167751300179675, -...",-1,0.547780
0,0,2019-12-24,"[-0.09616660106336444, -0.12656418912745335, -...",1,0.940841
0,0,2019-12-24,"[-0.09616660106336444, -0.12968012086338138, -...",1,0.937459
0,0,2019-12-24,"[-0.09616660106336444, -0.11865451625932838, -...",-1,0.995169


In [188]:
df['date']

0     2019-12-24
0     2019-12-24
0     2019-12-24
0     2019-12-24
0     2019-12-24
         ...    
13    2022-03-01
13    2022-03-01
13    2022-03-01
13    2022-03-01
13    2022-03-01
Name: date, Length: 2209150, dtype: object

In [189]:
df.inference.value_counts()

 1    1433980
-1     775170
Name: inference, dtype: int64

In [190]:
stock_df = pd.read_csv('./data/TSLA_2020_2022/stock_data/TSLA_2019-08-21_2022-05-12_2.csv')
stock_df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

In [191]:
stock_df.set_index('date', inplace=True)
stock_df.head()

,tslaopen,tslahigh,tslalow,tslaclose,close_rate
date,,,,,
2019-08-21,44.402000,44.644001,43.520000,44.166000,-0.042703
2019-08-22,44.560001,45.080002,43.644001,44.430000,-0.032185
2019-08-23,43.993999,44.234001,42.200001,42.279999,0.012677
2019-08-24,43.993999,44.234001,42.200001,42.279999,0.012677
2019-08-25,43.993999,44.234001,42.200001,42.279999,0.012677


In [192]:
df

,Unnamed: 0,date,features,inference,confidence
0,0,2019-12-24,"[-0.09616660106336444, -0.1308785561464306, -0...",1,0.938756
0,0,2019-12-24,"[-0.09616660106336444, -0.13167751300179675, -...",-1,0.547780
0,0,2019-12-24,"[-0.09616660106336444, -0.12656418912745335, -...",1,0.940841
0,0,2019-12-24,"[-0.09616660106336444, -0.12968012086338138, -...",1,0.937459
0,0,2019-12-24,"[-0.09616660106336444, -0.11865451625932838, -...",-1,0.995169
...,...,...,...,...,...
13,13,2022-03-01,"[-0.09616660106336444, -0.13047907771874753, -...",1,0.990381
13,13,2022-03-01,"[-0.09616660106336444, -0.12752293735389275, -...",1,0.998607
13,13,2022-03-01,"[-0.09616660106336444, -0.12968012086338138, -...",-1,0.991657
13,13,2022-03-01,"[-0.09616660106336444, -0.12904095537908844, -...",1,0.961570


In [193]:
stock_df.head()


,tslaopen,tslahigh,tslalow,tslaclose,close_rate
date,,,,,
2019-08-21,44.402000,44.644001,43.520000,44.166000,-0.042703
2019-08-22,44.560001,45.080002,43.644001,44.430000,-0.032185
2019-08-23,43.993999,44.234001,42.200001,42.279999,0.012677
2019-08-24,43.993999,44.234001,42.200001,42.279999,0.012677
2019-08-25,43.993999,44.234001,42.200001,42.279999,0.012677


In [194]:
df['close_rate'] =  df.apply(lambda x: stock_df.loc[x['date'], 'close_rate'], axis=1)


In [195]:
for i in range(10):
    df[f'feature_{i}'] = df.apply(lambda x: x['features'][i], axis=1)


In [200]:
nice_df = df.drop(columns=['features', 'Unnamed: 0'], axis=1)

In [204]:
infer_price = nice_df['inference'] * nice_df['confidence']


0     0.938756
0    -0.547780
0     0.940841
0     0.937459
0    -0.995169
        ...   
13    0.990381
13    0.998607
13   -0.991657
13    0.961570
13   -0.990270
Length: 2209150, dtype: float64

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# fig, axs = plt.subplots(1, 2, figsize=(20, 8))
# sns.histplot(data=nice_df, x="close_rate", ax=axs[0], kde=True)
# sns.histplot(data=nice_df, y="close_rate", ax=axs[1], bins=15)
# axs[0].set_title('Histplot1')
# axs[1].set_title('Histplot2')


In [209]:
nice_df.to_csv('./data/TSLA_2020_2022/all_in_one/features_normalized.csv')